In [9]:
import random
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# Variables globales para los datos
tiemposOperaciones = {
    'O1': [3.5, 6.7, 2.5, 8.2],
    'O2': [5.5, 4.2, 7.6, 9.0],
    'O3': [6.1, 7.3, 5.5, 6.7],
    'O4': [4.8, 5.3, 3.8, 4.7],
    'O5': [3.8, 3.4, 4.2, 3.6]
}

trabajos = {
    'j1': ['O2', 'O4', 'O5'],
    'j2': ['O1', 'O3', 'O5'],
    'j3': ['O1', 'O2', 'O3', 'O4', 'O5'],
    'j4': ['O4', 'O5'],
    'j5': ['O2', 'O4'],
    'j6': ['O1', 'O2', 'O4', 'O5']
}

# Configuraciones de parámetros del algoritmo genético que se usaran en ANOVA
configuraciones = [
    {'tamPoblacion': 20, 'generaciones': 300, 'tasaMutacion': 0.1, 'tasaCruce': 0.8},
    {'tamPoblacion': 100, 'generaciones': 150, 'tasaMutacion': 0.9, 'tasaCruce': 0.2},
    {'tamPoblacion': 300, 'generaciones': 500, 'tasaMutacion': 0.15, 'tasaCruce': 0.7},
    {'tamPoblacion': 10, 'generaciones': 80, 'tasaMutacion': 0.05, 'tasaCruce': 0.85},
    {'tamPoblacion': 500, 'generaciones': 10, 'tasaMutacion': 0.75, 'tasaCruce': 0.12}
]

# Función para generar una planificación aleatoria
def generarPlanificacion():
    planificacion = []
    for trabajo, operaciones in trabajos.items():
        asignacionMaquinas = [random.choice(range(1, len(tiemposOperaciones[operacion]) + 1)) for operacion in operaciones]
        planificacion.append((trabajo, asignacionMaquinas))
    return planificacion

# Función para evaluar el makespan de una planificación y guardar los tiempos de inicio y fin
def evaluarPlanificacion(planificacion):
    numero_maquinas = len(next(iter(tiemposOperaciones.values())))
    tiemposMaquinas = [0] * (numero_maquinas + 1)
    tiemposFinTrabajos = {trabajo: 0 for trabajo in trabajos.keys()} 
    tiemposTrabajos = [] 
    for trabajo, asignacionMaquinas in planificacion:
        tiempoInicioTrabajo = tiemposFinTrabajos[trabajo]
        for i, operacion in enumerate(trabajos[trabajo]):
            maquina = asignacionMaquinas[i]
            tiempoInicio = max(tiempoInicioTrabajo, tiemposMaquinas[maquina])
            tiempoFin = tiempoInicio + tiemposOperaciones[operacion][maquina-1]
            tiemposTrabajos.append((trabajo, operacion, maquina, tiempoInicio, tiempoFin))
            tiempoInicioTrabajo = tiempoFin
            tiemposMaquinas[maquina] = tiempoFin
        tiemposFinTrabajos[trabajo] = tiempoInicioTrabajo
    return max(tiemposMaquinas)

# Función de selección por torneo
def seleccionTorneo(poblacion, puntajes, k=2):
    seleccionados = random.sample(list(zip(poblacion, puntajes)), k)
    seleccionados.sort(key=lambda x: x[1])
    return seleccionados[0][0]

# Función de cruce (Cruza de un punto)
def cruce(padre1, padre2):
    punto = random.randint(1, len(padre1)-1)
    hijo1 = padre1[:punto] + padre2[punto:]
    hijo2 = padre2[:punto] + padre1[punto:]
    return hijo1, hijo2

# Función de mutación
def mutacion(planificacion):
    trabajo, asignacionMaquinas = random.choice(planificacion)
    indice = random.randint(0, len(asignacionMaquinas)-1)
    asignacionMaquinas[indice] = random.choice(range(1, len(tiemposOperaciones[trabajos[trabajo][indice]]) + 1))

# Algoritmo genético
def algoritmoGenetico(tamPoblacion, generaciones, tasaMutacion, tasaCruce):
    poblacion = [generarPlanificacion() for _ in range(tamPoblacion)]
    
    for generacion in range(generaciones):
        puntajesYtiempos = [evaluarPlanificacion(ind) for ind in poblacion]
        puntajes = [puntaje for puntaje in puntajesYtiempos]
        nuevaPoblacion = []
        for _ in range(tamPoblacion // 2):
            padre1 = seleccionTorneo(poblacion, puntajes)
            padre2 = seleccionTorneo(poblacion, puntajes)
            if random.random() < tasaCruce:
                hijo1, hijo2 = cruce(padre1, padre2)
            else:
                hijo1, hijo2 = padre1, padre2
            if random.random() < tasaMutacion:
                mutacion(hijo1)
            if random.random() < tasaMutacion:
                mutacion(hijo2)
            nuevaPoblacion.extend([hijo1, hijo2])
        poblacion = nuevaPoblacion

    mejorPuntaje = min(puntajes)
    return mejorPuntaje, tamPoblacion, generaciones, tasaMutacion, tasaCruce

# Función para ejecutar múltiples configuraciones y guardar resultados
def ejecutarConfiguraciones(configuraciones, num_ejecuciones=10):
    resultados = []
    for config in configuraciones:
        tiempos = []
        for _ in range(num_ejecuciones):
            random.seed()
            mejorPuntaje, tamPoblacion, generaciones, tasaMutacion, tasaCruce = algoritmoGenetico(
                config['tamPoblacion'], config['generaciones'], config['tasaMutacion'], config['tasaCruce']
            )
            tiempos.append(mejorPuntaje)
        resultados.append((tiempos, tamPoblacion, generaciones, tasaMutacion, tasaCruce))
    return resultados

# Función para realizar ANOVA y mostrar resultados
def realizarANOVA(resultados):
    makespans_por_configuracion = [result[0] for result in resultados]
    F, p = stats.f_oneway(*makespans_por_configuracion)
    print("Resultados del ANOVA:")
    print(f"Valor F: {F}")
    print(f"Valor p: {p}")

    # Análisis adicional de las configuraciones
    print("\nAnálisis de las configuraciones:")

    for i, config in enumerate(resultados):
        print(f"\nConfiguración {i+1}:")
        print(f" - Tamaño de población: {config[1]}")
        print(f" - Número de generaciones: {config[2]}")
        print(f" - Tasa de mutación: {config[3]}")
        print(f" - Tasa de cruce: {config[4]}")
        print(f" - Makespans promedio: {np.mean(config[0])}")
        print(f" - Desviación estándar: {np.std(config[0])}")

# Función principal
def main():
    # Ejecutar las configuraciones
    resultados = ejecutarConfiguraciones(configuraciones)

    # Realizar ANOVA y análisis de resultados
    realizarANOVA(resultados)

# Ejecutar el programa principal
if __name__ == "__main__":
    main()


Resultados del ANOVA:
Valor F: 12.315157040268868
Valor p: 7.5971194648822e-07

Análisis de las configuraciones:

Configuración 1:
 - Tamaño de población: 20
 - Número de generaciones: 300
 - Tasa de mutación: 0.1
 - Tasa de cruce: 0.8
 - Makespans promedio: 72.32
 - Desviación estándar: 8.344435271484826

Configuración 2:
 - Tamaño de población: 100
 - Número de generaciones: 150
 - Tasa de mutación: 0.9
 - Tasa de cruce: 0.2
 - Makespans promedio: 64.95
 - Desviación estándar: 9.729259992414633

Configuración 3:
 - Tamaño de población: 300
 - Número de generaciones: 500
 - Tasa de mutación: 0.15
 - Tasa de cruce: 0.7
 - Makespans promedio: 63.620000000000005
 - Desviación estándar: 12.169124865823342

Configuración 4:
 - Tamaño de población: 10
 - Número de generaciones: 80
 - Tasa de mutación: 0.05
 - Tasa de cruce: 0.85
 - Makespans promedio: 67.6
 - Desviación estándar: 11.630391222998478

Configuración 5:
 - Tamaño de población: 500
 - Número de generaciones: 10
 - Tasa de mutaci

Un valor F alto sugiere que las diferencias entre las configuraciones son significativas en comparación con las diferencias dentro de cada configuración.

El valor de p sugiere la probabilidad de obtener resultados tan extremos

En el problema:
Un valor F alto junto con un valor de p bajo generalmente sugiere que al menos una configuración del algoritmo genético está mostrando un rendimiento significativamente diferente en comparación con las demás configuraciones evaluadas, en términos del makespan. Esta configuracion como puede observarse es la configuracion 5